# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [3]:
#Your code here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

data = pd.read_csv('homepage_actions.csv')
print('data shape:', data.shape)
print(data.info())
data.head()

data shape: (8188, 4)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  8188 non-null   object
 1   id         8188 non-null   int64 
 2   group      8188 non-null   object
 3   action     8188 non-null   object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB
None


,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


How many viewers also clicked?

In [4]:
clicks = data[data['action'] == 'click']
views = data[data['action'] == 'view']

print("clicks: ", len(clicks))
print("views: ", len(views))

assert len(views) + len(clicks) == len(data)

clicks:  1860
views:  6328


Are there any anomalies with the data; did anyone click who didn't view?

In [5]:
cids = set(data[data.action=='click']['id'].unique())
vids = set(data[data.action=='view']['id'].unique())
print("Number of viewers: {} \tNumber of clickers: {}".format(len(vids), len(cids)))
print("Number of Viewers who didn't click: {}".format(len(vids-cids)))
print("Number of Clickers who didn't view: {}".format(len(cids-vids)))

Number of viewers: 6328 	Number of clickers: 1860
Number of Viewers who didn't click: 4468
Number of Clickers who didn't view: 0


Is there any overlap between the control and experiment groups?

In [6]:
eids = set(data[data.group=='experiment']['id'].unique())
cids = set(data[data.group=='control']['id'].unique())

print('Overlap of experiment and control groups: {}'.format(len(eids&cids)))

Overlap of experiment and control groups: 0


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [7]:
#Your code here
data['count'] = 1
data.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [8]:
control = data[data.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = data[data.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))

control = control.apply(lambda col: col.map(int))
experiment = experiment.apply(lambda col: col.map(int))

control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1,1
183089,0,1
183248,1,1
183515,0,1
183524,0,1


In [14]:
from scipy.stats import ttest_ind

# Assuming 'control' and 'experiment' DataFrames are already defined
control_clicks = control['click']
experiment_clicks = experiment['click']

# Perform a two-sample t-test
t_statistic, p_value = ttest_ind(control_clicks, experiment_clicks)

print("P-Value:", p_value)

P-Value: 0.008825098914958293


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [22]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)

print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [20]:
#Your code here
#Your code here
n = len(experiment)
p = control_rate
var = n * p * (1-p)
std = np.sqrt(var)

actual_experiment_clicks = experiment.click.sum()

z_score = (actual_experiment_clicks - expected_experiment_clicks_under_null)/std

print(z_score)

3.6625360854823588


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [23]:
#Your code here
import scipy.stats as stats
p_val = 1 - stats.norm.cdf(z_score)
print(p_val)

0.00012486528006949715


### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The z-score of 3.6625360854823588 and the p-value of 0.00012486528006949715 that i obtained from the Welch's t-test indicate a highly significant difference between the 'control' and 'experiment' groups for the 'click' variable.

Comparing this result to the previous statistical test with a p-value of 0.008825098914958293, i can see that the p-value from the Welch's t-test (0.00012486528006949715) is much smaller. A smaller p-value typically indicates stronger evidence against the null hypothesis, suggesting a more significant difference between the two groups.

In summary, the result from the Welch's t-test suggests a stronger statistical significance in the difference between the groups compared to the previous test with the higher p-value.**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.